In [1]:
import textstat
import pandas as pd
from ntk import Ntk

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import train_test_split




In [79]:
def load_gendered_names(shuffle: bool = True, gender_as_bool:bool=True):
    ntk = Ntk()

    m = [n for n in ntk.gutter if n not in ntk.jenter]
    f = [n for n in ntk.jenter if n not in ntk.gutter]

    names = m + f
    genders = ['Male' for n in m] + ["Female" for n in f]

    names_df = pd.DataFrame([(k, v) for k, v in zip(
        names, genders)], columns=['Name', 'Gender'])


    if shuffle:
        names_df = names_df.sample(
            frac=1, random_state=42).reset_index(drop=True)

    #
    if gender_as_bool:
        # swap "Mann", "Kvinne" with 1, 0
        from sklearn.preprocessing import LabelBinarizer
        lb = LabelBinarizer()

        binary_gender = lb.fit_transform(names_df['Gender'])
        print(lb.inverse_transform(binary_gender))
        ##target_strings = lb.inverse_transform(np.arange(num_classes))

        names_df['Gender'] = binary_gender

    return names_df

load_gendered_names().head()

['Female' 'Male' 'Female' ... 'Female' 'Female' 'Male']


,Name,Gender
0,Váreyð,0
1,Albèrto,1
2,Alannah,0
3,Nicola,0
4,Cathrine,0


In [5]:


class DataframeFunctionTransformer():
    """Class that allows me to apply a function to my df. 
    Like drop_cols and generate_language_features"""
    def __init__(self, func):
        self.func = func

    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self

from aux_functions import drop_cols, generate_language_features


In [6]:
from sklearn.base import TransformerMixin, BaseEstimator
class Debug(BaseEstimator, TransformerMixin):
    """Class only used to debug sklearn Piplines """    
    def transform(self, X):
        print("Debug transform")
        print(X)
        print(pd.DataFrame(X).head())
        print(X.shape)
        return X

    def fit(self, X, y=None, **fit_params):
        return self

In [48]:



# ColumnTransformer applies function (ohe) to each of the columns listed after it ("name", Funcion, [...])
preprocessor = ColumnTransformer(
    transformers=[
        ("one_hot", OneHotEncoder(handle_unknown="ignore"), ["last_1_letter", "last_2_letter", "last_3_letter"]),
    ],
    remainder='passthrough'
)

import lightgbm as lgb
import xgboost as xgb

from xgboost import XGBClassifier


from sklearn.ensemble import GradientBoostingClassifier




# this pipeline has a single step
pipeline_engineer_transform = Pipeline(
    verbose=True,
    steps=[
    ("generate_features", DataframeFunctionTransformer(generate_language_features)),
   # ('dbg', Debug()),
    ("drop_cols", DataframeFunctionTransformer(drop_cols)),
    ("ohe_cat_variables", preprocessor),
    # ,
    # ("model", lgb.LGBMClassifier(colsample_bytree=0.5205191706285026, learning_rate=1.0,
    #            max_bin=511, min_child_samples=2, n_estimators=42, num_leaves=10,
    #            reg_alpha=0.0922808777931273, reg_lambda=0.0009765625,
    #            verbose=-1))
    # ("clf", XGBClassifier(base_score=0.5, booster='gbtree',
    #           colsample_bylevel=0.8208880615640181, colsample_bynode=1,
    #           colsample_bytree=0.46403642305886156, gamma=0, gpu_id=-1,
    #           grow_policy='lossguide', importance_type='gain',
    #           interaction_constraints='', learning_rate=0.05003731635910742,
    #           max_delta_step=0, max_depth=0, max_leaves=3068,
    #           min_child_weight=0.06336777978022778, missing=None,
    #           monotone_constraints='()', n_estimators=18, n_jobs=-1,
    #           num_parallel_tree=1, random_state=0,
    #           reg_alpha=0.0030138724246546837, reg_lambda=0.37283300242426975,
    #           scale_pos_weight=1, subsample=0.8061527641993345,
    #           tree_method='hist', use_label_encoder=False,
    #           validate_parameters=1, verbosity=0))
])


In [49]:
# Lets load data



df = load_gendered_names()
df.head(2)


,Name,Gender
0,Váreyð,0
1,Albèrto,1


In [58]:
y = df['Gender']
X = df.drop(columns=['Gender']) # Name


X = pipeline_engineer_transform.fit_transform(X)
X

tranformed_df = pd.DataFrame.sparse.from_spmatrix(
    X, #pipeline.transform(pd.DataFrame({"Name": ["Eirik"]})), 
    columns=pipeline_engineer_transform[2].get_feature_names_out()
    )

tranformed_df.head()

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=99)


pd.DataFrame.sparse.from_spmatrix(
    X_train, #pipeline.transform(pd.DataFrame({"Name": ["Eirik"]})), 
    columns=pipeline_engineer_transform[2].get_feature_names_out()
    ).head()
#_ = pipeline.fit_transform(X_train, y=y_train)


[Pipeline] . (step 1 of 3) Processing generate_features, total=   0.1s
[Pipeline] ......... (step 2 of 3) Processing drop_cols, total=   0.0s
[Pipeline] . (step 3 of 3) Processing ohe_cat_variables, total=   0.0s


<5505x1865 sparse matrix of type '<class 'numpy.float64'>'
	with 24548 stored elements in Compressed Sparse Row format>

,one_hot__last_1_letter_*,one_hot__last_1_letter_A,one_hot__last_1_letter_B,one_hot__last_1_letter_C,one_hot__last_1_letter_D,one_hot__last_1_letter_E,one_hot__last_1_letter_F,one_hot__last_1_letter_G,one_hot__last_1_letter_H,one_hot__last_1_letter_I,one_hot__last_1_letter_J,one_hot__last_1_letter_K,one_hot__last_1_letter_L,one_hot__last_1_letter_M,one_hot__last_1_letter_N,...,one_hot__last_3_letter_ØRK,one_hot__last_3_letter_ØRN,one_hot__last_3_letter_ØRT,one_hot__last_3_letter_ØVE,one_hot__last_3_letter_ØYA,one_hot__last_3_letter_ÚLI,one_hot__last_3_letter_ÚNA,one_hot__last_3_letter_ÚNI,one_hot__last_3_letter_ÚSI,one_hot__last_3_letter_ÚST,one_hot__last_3_letter_ÝR,one_hot__last_3_letter_ÞÓR,one_hot__last_3_letter_ﾘRN,remainder__final_letter_is_vowel,remainder__syllable_count
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


,one_hot__last_1_letter_*,one_hot__last_1_letter_A,one_hot__last_1_letter_B,one_hot__last_1_letter_C,one_hot__last_1_letter_D,one_hot__last_1_letter_E,one_hot__last_1_letter_F,one_hot__last_1_letter_G,one_hot__last_1_letter_H,one_hot__last_1_letter_I,one_hot__last_1_letter_J,one_hot__last_1_letter_K,one_hot__last_1_letter_L,one_hot__last_1_letter_M,one_hot__last_1_letter_N,...,one_hot__last_3_letter_ØRK,one_hot__last_3_letter_ØRN,one_hot__last_3_letter_ØRT,one_hot__last_3_letter_ØVE,one_hot__last_3_letter_ØYA,one_hot__last_3_letter_ÚLI,one_hot__last_3_letter_ÚNA,one_hot__last_3_letter_ÚNI,one_hot__last_3_letter_ÚSI,one_hot__last_3_letter_ÚST,one_hot__last_3_letter_ÝR,one_hot__last_3_letter_ÞÓR,one_hot__last_3_letter_ﾘRN,remainder__final_letter_is_vowel,remainder__syllable_count
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0


In [81]:

clf = XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.8208880615640181, colsample_bynode=1,
              colsample_bytree=0.46403642305886156, gamma=0, gpu_id=-1,
              grow_policy='lossguide', importance_type='gain',
              interaction_constraints='', learning_rate=0.05003731635910742,
              max_delta_step=0, max_depth=0, max_leaves=3068,
              min_child_weight=0.06336777978022778, missing=None,
              monotone_constraints='()', n_estimators=18, n_jobs=-1,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0.0030138724246546837, reg_lambda=0.37283300242426975,
              scale_pos_weight=1, subsample=0.8061527641993345,
              tree_method='hist', use_label_encoder=False,
              validate_parameters=1, verbosity=0)

clf.fit(X_train, y_train)

predicted = clf.predict(X_val)
clf.score(X_val, y_val)

from sklearn.metrics import classification_report
print(classification_report(y_val, predicted, target_names=['0','1'])) # female/male? 


#_.score(X_val, y_val)
#_[-2].get_feature_names_out()

XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.8208880615640181, colsample_bynode=1,
              colsample_bytree=0.46403642305886156, gamma=0, gpu_id=-1,
              grow_policy='lossguide', importance_type='gain',
              interaction_constraints='', learning_rate=0.05003731635910742,
              max_delta_step=0, max_depth=0, max_leaves=3068,
              min_child_weight=0.06336777978022778, missing=None,
              monotone_constraints='()', n_estimators=18, n_jobs=-1,
              num_parallel_tree=1, random_state=0,
              reg_alpha=0.0030138724246546837, reg_lambda=0.37283300242426975,
              scale_pos_weight=1, subsample=0.8061527641993345,
              tree_method='hist', use_label_encoder=False,
              validate_parameters=1, verbosity=0)

0.8460421205519245

              precision    recall  f1-score   support

           0       0.85      0.83      0.84       669
           1       0.84      0.86      0.85       708

    accuracy                           0.85      1377
   macro avg       0.85      0.85      0.85      1377
weighted avg       0.85      0.85      0.85      1377



In [84]:
# transform new data and predict that

new_df = pd.DataFrame({"Name": ['Eirika', 'Kathasarina', 'Anders', 'Sindre']})

new_features = pipeline_engineer_transform.transform(new_df)

pd.DataFrame.sparse.from_spmatrix(
    new_features, 
    columns=pipeline_engineer_transform[2].get_feature_names_out()
    ).head()


clf.predict(new_features)

clf.predict_proba(new_features)

labels = ['female', 'male']

new_df['pred'] = list(map(lambda n: labels[n], clf.predict(new_features)))

new_df

,one_hot__last_1_letter_*,one_hot__last_1_letter_A,one_hot__last_1_letter_B,one_hot__last_1_letter_C,one_hot__last_1_letter_D,one_hot__last_1_letter_E,one_hot__last_1_letter_F,one_hot__last_1_letter_G,one_hot__last_1_letter_H,one_hot__last_1_letter_I,one_hot__last_1_letter_J,one_hot__last_1_letter_K,one_hot__last_1_letter_L,one_hot__last_1_letter_M,one_hot__last_1_letter_N,...,one_hot__last_3_letter_ØRK,one_hot__last_3_letter_ØRN,one_hot__last_3_letter_ØRT,one_hot__last_3_letter_ØVE,one_hot__last_3_letter_ØYA,one_hot__last_3_letter_ÚLI,one_hot__last_3_letter_ÚNA,one_hot__last_3_letter_ÚNI,one_hot__last_3_letter_ÚSI,one_hot__last_3_letter_ÚST,one_hot__last_3_letter_ÝR,one_hot__last_3_letter_ÞÓR,one_hot__last_3_letter_ﾘRN,remainder__final_letter_is_vowel,remainder__syllable_count
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0


array([0, 0, 1, 1])

array([[0.76212114, 0.23787887],
       [0.79208577, 0.2079142 ],
       [0.35906732, 0.6409327 ],
       [0.35418004, 0.64581996]], dtype=float32)

,final_letter_is_vowel,syllable_count,last_1_letter,last_2_letter,last_3_letter,pred
0,1,2,A,KA,IKA,female
1,1,3,A,NA,INA,female
2,0,2,S,RS,ERS,male
3,1,2,E,RE,DRE,male
